# Assignment 01 - Pitfalls in Data Mining     
## CSCI E-96

The goal of data mining is to find important relationships in large complex datasets. These dataset typically contain a large number of variables. The **high-dimensional** nature of the data leads to some commonly encountered pitfalls which lead to incorrect inferences.   

In this assignment you will gain a bit of experience with three important concepts in data mining:  

1. **False Discovery Rate Control:** The goal of data mining is to find important relationships in large complex datasets. These dataset typically contain a large number of variables. The **high-dimensional** nature of the data leads to some commonly encountered pitfalls which lead to incorrect inferences. A related problem is cutting off a large-scale analysis when a desired relationship is 'found'. This practice of **p-value mining** often leads to unwarranted inferences. You will apply false discovery rate (FDR) control methods to address this problem.   
2. **Key-Value Pairs:** Large scale data is typically managed using key-value (KV) pairs. The exercises in this assignment give you some experience working with KV pair data management.  
3. **Map and Reduce Processes:** Much of large scale data mining requires use of a split-apply-combine approach. The data is split into manageable chunks, analytic transformations are applied, and the result combined or aggregated. A commonly used class of a split-apply-combine algorithm is MapReduce. 

In order to keep the scope of this assignment manageable, you will use limited versions of KV pair management and MapReduce. Specifically, you will use common Python tools to implement these concepts rather than dedicated large scale analytic platforms. 

## Multiple Hypothesis Tesing

Testing multiple hypothesis in high-dimensional data can be problematic. Exhaustively testing all pairwise relationships between variables in a data set is a commonly used, but generally misleading from of **multiple comparisons**. The chance of finding false significance, using such a **data dredging** approach, can be surprisingly high. 

In this exercise you will perform multiple comparisons on only 20 **identically distributed independent (iid)** variables. Ideally, such tests should not find significant relationships, but the actual result is quite different. 

To get started, execute the code in the cell below to load the required packages. 

In [9]:
import pandas as pd
import numpy as np
import numpy.random as nr
from scipy.stats import ttest_ind, f_oneway
from itertools import product, combinations
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.multitest as smt

pd.set_option("display.max_rows", None, "display.max_columns", None)

In this exercise you will apply a t-test to all pairwise combinations of identical Normally distributed variables. In this case, we will create a data set with 20 iid Normal distributions of 1000 samples each. Execute the code in the cell below to find this data and display the mean and variance of each variable.  

In [10]:
ncolumns = 20
nr.seed(234)
mean = []
var = []
ttest = []
normal_vars = nr.normal(size=(1000,ncolumns))
print('NORMAL VARS')
print(normal_vars)
print('Len of normal vars')
print(len(normal_vars))
print('The means of the columns are\n', np.mean(normal_vars, axis = 0))
print('\nThe variances of the columns are\n', np.var(normal_vars, axis = 0))
##create arrays for mean and var
for mn in np.mean(normal_vars, axis =0 ):
    mean.append(mn)
print(mean)
    
for vr in np.var(normal_vars, axis = 0):
    var.append(vr)
print(var)

NORMAL VARS
[[ 0.81879162 -1.04355064  0.3509007  ... -0.17542066 -0.08270438
  -0.88845473]
 [-0.30076649  0.90837517 -0.64559131 ... -0.46415036 -0.79785911
   1.31076281]
 [ 1.17479389 -0.05046953  0.71895176 ...  0.71999964 -0.86680023
  -0.34668516]
 ...
 [ 0.0943446  -0.88593117  2.21144516 ... -0.48951209 -0.14650596
   1.468948  ]
 [-0.99396682 -0.60068401 -1.06598225 ...  0.80637671 -0.38532756
  -0.06067443]
 [-0.7191216   0.0248463   0.2728283  ...  0.3302862   0.2384886
   0.98507076]]
Len of normal vars
1000
The means of the columns are
 [-1.16191649e-01  2.80829317e-02 -1.78516419e-02 -1.44691489e-02
  3.03718152e-02  1.20007442e-02 -9.58845606e-05  1.98662580e-03
  4.94154934e-02 -4.11640866e-02 -6.32977862e-03 -5.93868192e-02
 -2.56373595e-02  1.43568791e-02 -1.44725765e-02 -1.37023955e-02
  1.80622439e-02  5.87029691e-02 -2.02650514e-02 -1.56346106e-02]

The variances of the columns are
 [0.94834508 1.04744241 1.0258018  0.96977571 1.0089001  1.04113864
 1.00657222 0.9

Notice that means and variances are close to 0.0 and 1.0 respectively. As expected, there is not much difference between these variables.

How many of these t-tests will show **significance** at the 0.05 cut-off level? There are 380 pairwise combinations, so we expect to find a number of falsely significant test results at this level. To find out, complete and execute the code in the cell below to filter the test results and print those that show significance. 

### Creating a hash 

The goal of this exercise is to compute pairwise hypothesis tests of the differences in means for each of the iid Normal vectors. As an intermediate step you will create a Pandas data frame using a hash of the keys of the vectors. The data frame will contain the **key-value**, $(K,V)$, pairs. Each key must represent an index for the two vectors used to compute the test statistic. The keys will then be used to index the results of these hypothesis tests. 

The question is, how can we create a hash from the keys for the pair of vectors? In this case to we will use a simple, but far from optimal hash. For the two vector indicies $i, j$, for some key and modulo, $m$, we will compute the hash as:  

$$h(i,j) = (i + key*j) mod m$$

> **Computational Note:** The Pandas data frame is an efficient and reasonably scalable **hash table**. The hash function used depends on the type of the key; integer, string, etc. The resulting dictionary of key-value pairs, $(K,V)$, can therefore be access in far less than linear time, often about $O(log(N))$.  

If you are not familiar with Python dictionaries you can find a short tutorial [here](https://www.tutorialspoint.com/python_data_structure/python_hash_table.htm), as well as many other places on the web.

> **Exercise 1-1:** Given that our space of vectors is actually quite small, just 20, we do not need a sophisticated and scalable hash function. This hashed key will then be used to store and retrieve the values using a Python dictionary, in about $O(log(N))$ time.     

> In this exercise you will test a simple hash function and its inverse. Examine the code below and notice that the hash function encodes the two indexes into a single integer by simple additional and multiplication. The modulo operation limits the size of the hash table. However, to keep things simple you will not need to implement any hash collision resolution mechanism. As a result, the size of the table is set much larger than required.  

> To test this hash, do the following:    
> 1. Create a function called hash function to compute the hash. The arguments to the function are $i$ and $j$, the `hash\_key` and the `modulo\_multiplier`. The defaults of the arguments are $hash\_key=1024$ and $modulo\_multiplier=32$. The modulo number is $hash\_key * modulo\_multiplier$, e.g. $modulo = 32,768$. The multiplier is the ratio of expected values stored, $n$, to the number of unique hash keys, $m$, e.g. the ratio $m/n$.
> 2. Using the Python [ittertools.combinations](https://docs.python.org/3/library/itertools.html#itertools.combinations) function create all unique pairwise combinations of indexes i and j. The arguments to this function are the indexes to the iid Normal vectors. The iterator is `range(ncolumns)` choose 2, since these comparisons are pairwise.    
> 3. Within this loop call the hash with the values of $i$ and $j$ as arguments.   
> 3. On a single line print the following; the values of i and j, the hash key value, but only if $i \le 6$. The restriction is to keep the printed output shorter.   

In [11]:
def hash_function(i, j, hash_key=1024, modulo_multiplier=32):
    ## Put your code below. 
    ##return ((i + hash_key) * j) % modulo_multiplier
    return  ((i + hash_key) * j) % (hash_key*modulo_multiplier)

count =0
hash = {}
harr =  []
    ##h[(i,j)] = ((i + hash_key) * j ) % modulo_multiplier

for i,j in combinations(range(ncolumns), 2):
    if i <= 6: 
        
        print( ' Count = ' + str(count))
        ##hash[(i,j)] = hash_function(i,j)
    hash[(i,j)] =     hash_function(i,j)
    harr.append( hash_function(i,j))

        ##print('i = ' + str(i) + '  j = ' + str(j) + '   hash = ' + str(hash) + '     Count hash = ' + str(count+1))
    count += 1
    combCount = 1
        
for i,j in combinations(range(ncolumns),2):     
    if i<= 6:
        print('COUNT: '+ str(combCount) + ':  hash[(' + str(i)+ ' , '+ str( j) + ')] : ' + str(hash[i,j]))
    combCount += 1
        
print(' The Total Number of Combinations is: ' + str(count))
    
Count1 = 0  
harr_u = np.unique(harr)
for u in harr_u:
    
    print('harr_unique(' + str(Count1) + '): ' + str(u) + ' COUNT: '+ str(Count1+1))
    Count1 += 1
## print (harr_u)
## print('harr_unique(' + str(Count1-1) + '): ' + str(u) + ' COUNT: '+ str(Count1))
print(' The Total Number of unique instances is: '+ str(Count1))



 Count = 0
 Count = 1
 Count = 2
 Count = 3
 Count = 4
 Count = 5
 Count = 6
 Count = 7
 Count = 8
 Count = 9
 Count = 10
 Count = 11
 Count = 12
 Count = 13
 Count = 14
 Count = 15
 Count = 16
 Count = 17
 Count = 18
 Count = 19
 Count = 20
 Count = 21
 Count = 22
 Count = 23
 Count = 24
 Count = 25
 Count = 26
 Count = 27
 Count = 28
 Count = 29
 Count = 30
 Count = 31
 Count = 32
 Count = 33
 Count = 34
 Count = 35
 Count = 36
 Count = 37
 Count = 38
 Count = 39
 Count = 40
 Count = 41
 Count = 42
 Count = 43
 Count = 44
 Count = 45
 Count = 46
 Count = 47
 Count = 48
 Count = 49
 Count = 50
 Count = 51
 Count = 52
 Count = 53
 Count = 54
 Count = 55
 Count = 56
 Count = 57
 Count = 58
 Count = 59
 Count = 60
 Count = 61
 Count = 62
 Count = 63
 Count = 64
 Count = 65
 Count = 66
 Count = 67
 Count = 68
 Count = 69
 Count = 70
 Count = 71
 Count = 72
 Count = 73
 Count = 74
 Count = 75
 Count = 76
 Count = 77
 Count = 78
 Count = 79
 Count = 80
 Count = 81
 Count = 82
 Count = 83
 C

> Examine the key pairs and the hash values. The question is, are there any hash collisions? This can be done as follows:   
> 5. Compute a list of the hash values for all combinations of $i$ and $j$.   
> 6. Print the length of the list.  
> 7. Print the length of the unique values of the hash. You can find the unique values in a list with the [numpy.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) function. 

In [12]:
## Put your code below. 

Count1 = 0
ui, harr_u = np.unique(harr,return_counts = True)

for uindex in range(len(harr_u)):
    Count1 += 1
    print('harr_unique(' + str(uindex) + '): ' + str(ui[uindex]) + ' Reoccurrances: '+ str(harr_u[uindex]))

print(' The Total Number of unique instances is: '+ str(Count1))

##print (ui)
##print (harr_u)

harr_unique(0): 1024 Reoccurrances: 1
harr_unique(1): 2048 Reoccurrances: 1
harr_unique(2): 2050 Reoccurrances: 1
harr_unique(3): 3072 Reoccurrances: 1
harr_unique(4): 3075 Reoccurrances: 1
harr_unique(5): 3078 Reoccurrances: 1
harr_unique(6): 4096 Reoccurrances: 1
harr_unique(7): 4100 Reoccurrances: 1
harr_unique(8): 4104 Reoccurrances: 1
harr_unique(9): 4108 Reoccurrances: 1
harr_unique(10): 5120 Reoccurrances: 1
harr_unique(11): 5125 Reoccurrances: 1
harr_unique(12): 5130 Reoccurrances: 1
harr_unique(13): 5135 Reoccurrances: 1
harr_unique(14): 5140 Reoccurrances: 1
harr_unique(15): 6144 Reoccurrances: 1
harr_unique(16): 6150 Reoccurrances: 1
harr_unique(17): 6156 Reoccurrances: 1
harr_unique(18): 6162 Reoccurrances: 1
harr_unique(19): 6168 Reoccurrances: 1
harr_unique(20): 6174 Reoccurrances: 1
harr_unique(21): 7168 Reoccurrances: 1
harr_unique(22): 7175 Reoccurrances: 1
harr_unique(23): 7182 Reoccurrances: 1
harr_unique(24): 7189 Reoccurrances: 1
harr_unique(25): 7196 Reoccurrances

> Examine the results you have printed. Is there any evidence of hash key collisions?     
> The ratio of $m/n$ is deliberately kept high since the simple hash function has no collision resolution mechanism. Optionally, you can try reducing this ration (the multiplier) to 16 and 8, noting the increase in hash collisions.  
> **End of exercise.**

No: There is no evidence of hash key collisions. There are 190 Combinations with 190 unique values. Each reoccurance has been determined for the unique hash values indicating the number of collisions.

### The map process

We are constructing this example a map and a reduce process. The processes are intended to compute the hypothesis test for differences of means between all the pairs of vectors. The first step is the map process, which creates the keys, or values of $i$ and $j$ for these pairs.   

> **Exercise 1-2:** You will now create the code for the map task which build a data frame with $i, j$ key pairs indexed by the hash. By the following steps you will create code that represents a map task.  
> 1. Create a data frame with two columns $i$ and $j$ with rows $= hash_key * modulo_multiplier $ and set all values to $= numpy.nan$.
> 2. Create a loop over all combinations of the pairs of i and j.   
> 2. Compute the hash key value for the indexes, i and j.  
> 4. Add the $i$ and $j$ values to the row indexed by the hash key.  
> 5. Return the hash table. 
> 6. Execute the function to create the hash table.  
> 7. Compute and print the length of the hash table. 

In [13]:
def map_hypothesis(vars, hash_key=1024, modulo_multiplier=32):


    ## Put your code below. 
    print('VARS ARE HERE: 1000 rows with 20 columns')
    print(vars)
    print('VARS Rows')
    print(len(vars))
    print('SHAPE Rows')
    print(vars.shape)
    arr = [[], []]
    arr_vals = []
    hash_arr = []
    arr = [[0,0],
           [0,1],
           [1,0],
           [1,1],
           [1,2]]
    lenrow = len(arr)
    lencol = len(arr[0])
    ind = 0
    ##for x in range(lenrow):
    ##    for y in range(lencol):
    ##        arr_vals[ind]= [(i),(j),np.NAN]
    ##        ind += 1
    
    ##ncolumns = 20
    ##nr.seed(234)
    ##normal_vars = nr.normal(size=(1000,ncolumns))
    
    
    
    ## make the index or rows
    df = pd.DataFrame(arr)
    
    ##df.index = hash_key * modulo_multiplier
    df.columns = ['i','j']
    print(df)
    ##df.loc = pd.columns
    ##print(df)
    ncols = vars.shape[1]
    ncols1 = len(vars[0])
    ##ij_val = [[],[]]
    j_val = []
    hash_val = []
    ncolumns = ncols
    print('ncols-shape :' + str(ncols) + ' ncols1-len(vars[0]) :' + str(ncols1))
    ##ncls = 20
    ## J goes from 0 to 19
    ind = 0
    for i,j in combinations(range(ncols), 2):
        
        
        ##ij_val[i,j] = vars[i,j]
        
        
        ind += 1
        
        ##hash_function(i, j, hash_key=1024, modulo_multiplier=32):
        ## The hash is actually the values of the columns
        hash[(i,j)] = hash_function(i,j)
        hash_arr.append(hash_function(i,j))
        hash_val.append( hash[(i,j)]*modulo_multiplier)
        newRow = (i,j,hash[i,j]*modulo_multiplier)
        arr_vals.append(newRow)
        ##arr[i,j] = np.nan
        ##df.append(new_row, ignore_index = True)
        ##arr[int(str(i)),int(str(j))] = np.nan
        
        ## Put your code below. 
        ## Compute the hash key and added the values to the 
        ## row of the data frame. 
    ##print(df.loc[:,'i','j'])
    print (ind)
    df_arr = pd.DataFrame(arr_vals)
   
    df_arr.columns = ['i','j', 'hash']
    df_arr.loc[:,'hash'] = pd.DataFrame(hash_val)
    combCount = 1
    ##for x,y in combinations(range(ncols), 2):
    ##    hash_arr.append(hash_function(x,y))
    ##    print('COUNT: '+ str(combCount) + ':  hash[(' + str(x)+ ' , '+ str( y) + ')] : ' + str(hash_function(x,y)))
    ##df_arr.loc[:,'hash'] = hash_arr
    df_arr.index = hash_arr
    print('DF_ARR')
    print(df_arr)
    return df_arr 
    ##return hash_table
    ##return print(normal_vars)

##normal_vars = nr.normal(size=(1000,ncolumns))
print('normal_vars')
print(normal_vars)
hash_table = map_hypothesis(normal_vars)
print('Length of Hash Table is: '+ str(len(hash_table)))

normal_vars
[[ 0.81879162 -1.04355064  0.3509007  ... -0.17542066 -0.08270438
  -0.88845473]
 [-0.30076649  0.90837517 -0.64559131 ... -0.46415036 -0.79785911
   1.31076281]
 [ 1.17479389 -0.05046953  0.71895176 ...  0.71999964 -0.86680023
  -0.34668516]
 ...
 [ 0.0943446  -0.88593117  2.21144516 ... -0.48951209 -0.14650596
   1.468948  ]
 [-0.99396682 -0.60068401 -1.06598225 ...  0.80637671 -0.38532756
  -0.06067443]
 [-0.7191216   0.0248463   0.2728283  ...  0.3302862   0.2384886
   0.98507076]]
VARS ARE HERE: 1000 rows with 20 columns
[[ 0.81879162 -1.04355064  0.3509007  ... -0.17542066 -0.08270438
  -0.88845473]
 [-0.30076649  0.90837517 -0.64559131 ... -0.46415036 -0.79785911
   1.31076281]
 [ 1.17479389 -0.05046953  0.71895176 ...  0.71999964 -0.86680023
  -0.34668516]
 ...
 [ 0.0943446  -0.88593117  2.21144516 ... -0.48951209 -0.14650596
   1.468948  ]
 [-0.99396682 -0.60068401 -1.06598225 ...  0.80637671 -0.38532756
  -0.06067443]
 [-0.7191216   0.0248463   0.2728283  ...  0.3

> **End of exercise.**

### The shuffle and reduce task

Now that you have the keys for the pairwise combinations of the vectors it is time to perform the reduce process. The reduce process computes the pair-wise t-statistics and p-values. These statistical values are indexed by the keys of the pair of vectors. This process reduces the full vectors of values down to just two numbers for each pair of vectors. 

> **Exercise 1-3:** You will now create and apply the following code for the reduce process:   
> 1. Create an empty data frame with columns, `i`, `j`, `t_statistic`, and `p_value`.    
> 2. Using a for loop iterate over all possible (hashed) keys of the data frame. An if statement is used to test if these are valid values of the key, i. Use the [numpy.isnan](https://numpy.org/doc/stable/reference/generated/numpy.isnan.html) function for this test.  
> 3. Extract the values of i and j from the input data frame. 
> 4. Using keys, compute the t-statistic and p-value using [scipy.stats import ttest_ind](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html).
> 5. Append a row to the output data frame.
> 6. Return the data frame, sorted in ascending order, using the [Pandas.DataFrame.sort_values](https://turned.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) method and re-indexed using the [Pandas.DataFrame.reset_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) method.    
> 7. Execute your function and save the returned data frame.

In [26]:
def reduce_significance(hash_table, values):  
    ## Create a data framreturn the results of the 
    ## the reduce process. The results are grouped by the first 
    ## index i. 
    print('HASH TABLE')
    ##print(hash_table)
   
    ncols = hash_table.shape[1]
    nrows = hash_table.shape[0]
    ##tt =ttest_ind(normal_vars[:,0], nr.normal(loc = 0.01, size=(1000,1)))
    tt =ttest_ind(normal_vars[:,0], nr.normal(loc = 0.01, size=(1000,1)))
    print( tt)
    ##test_results = pd.DataFrame(columns=['i','j','t_statistic','p_value'])
    test_results = pd.DataFrame(columns=['i','j'])
    ##print('TEST_RESULTS: ' + test_results)
    ## As a substitute for a shuffle we will use a simple search 
    ## through the data frame  
    count2 = 0
    ##print(  hash_table.loc[:,:'j']) 
       ## print[hindex]
    ## Put your code below. 
    extract_i = []
    extract_j = []
    ##for hash in range(hash_table.shape[0]): 
    ##    if not np.isnan(hash_table.iloc[hash,0]):
    ##        extract_i.append(hash_table['i'][hash])
    ##        extract_j.append(hash_table['j'][hash])
            ##print(hash1)
    ##        count2 += 1
    i_arr = []
    j_arr = []
    data = []
    hash_arr  = []
    tt_arr = []
    pv_arr = []
    sig_arr = []
    sig_count = 0
    ## ^^^
    ##print(' NCOLUMNS =' + str(ncolumns))
    ## 
    for i,j in combinations(range(ncolumns), 2):
        i_arr.append(i)
        j_arr.append(j)
        
        hash_arr = hash_function(i,j)
        
        data.append((i,j,hash_function(i,j)))
        
        tt = ttest_ind(values[:,i] , values[:,j]).statistic
        ##print(tt)
        tt_arr.append(tt)
        pv = ttest_ind(values[:,i] , values[:,j]).pvalue
        
        if pv <= 0.05:
            sig_arr.append('SIG')
            sig_count += 1
        else:
            sig_arr.append('Not-Sig')
        pv_arr.append(pv)
        ##print('SIG-COUNT: ' + str(sig_count))
        ##data.append((i,j))
        ##tt = ttest_ind(values.iloc[][i] , values.iloc[][j]).statistic
        

    ##print('PRINT h')
    print('THIS IS THE data')
    print(data)
    #test_results = pd.DataFrame(data)
    
    for hash in range(hash_table.shape[0]): 
        if not np.isnan(hash_table.iloc[hash,0]):
            h = 0
            ##tt = ttest_ind(hash_table.iloc[hash][1] , hash_table.iloc[hash][2])
            ##tt_arr.append(tt)
            ##pv = ttest_ind(hash_table.iloc[hash][1] , hash_table.iloc[hash][2]).pvalue
            ##pv_arr.append(pv)
            
    ##        extract_i.append(hash_table['i'][hash])
    ##        extract_j.append(hash_table['j'][hash])
    
   ## for h in range(len(hash_table.index)):
   ##     tt = ttest_ind(h['i'][h] , ttest_ind(h['j'][h]))
   ##     tt_ar.append(tt)
   ##    pv = ttest_ind(h['i'][h] , ttest_ind(h['j'][h])).pvalue
   ##     pv_arr.append(pv)
        ##print(h)
        
    
    print('len(hash_table): ' + str(len(hash_table)) )  
    print('len(test_results): ' + str(len(test_results)) ) 
    print('len(tt_arr): '+ str(len(tt_arr)))
    print('len(pv_arr): '+ str(len(pv_arr)))
    print('len(sig_arr): '+ str(len(sig_arr)))
    print('Sig-Count: ' + str(sig_count))
    test_results = pd.DataFrame(data)
    test_results.columns=['i','j','hash']
    test_results.loc[:,'t_statistic'] = tt_arr
    test_results.loc[:,'p_value'] = pv_arr
    test_results.loc[:,'sig'] = sig_arr
    print('PRINT TEST_RESULTS')
    print(test_results)
    #test_results[:,'i'] = pd.DataFrame(hash_table.loc[:,'i'])
    #df_j = pd.DataFrame(hash_table.loc[:,'j'])
    #print('df_i : ' + df_i)
    #print(df_j)
    #print('BEFORE .values')
    #print(df_i.values)
    #df_i_1D = []
    #df_j_1D = []
    #for item in df_i.values:
    #    df_i_1D.append(df_i.values[0])
    #for itemj in df_j.values:
    #    df_j_1D.append(df_i.values[0])
    ##print('DTYPE: ' + np.dtype(df_i)
    #test_results.loc[:,'i'] = df_i_1D
    #test_results.loc[:,'j'] = df_j_1D
    ##print('TEST RESULTS II: ' + test_results)
    ## Given the i,j pairs we need to compute the t-statistic and the p-value.   
    ## This is the reduce step, since for each i,j pair there is only 
    ## a t-statistic and a p-value. 
    ## Put your code below. 
           
    ## Sort and return the results
    #test_results.sort_values('p_value', axis=0, ascending=True# , inplace=True)
    return test_results.sort_values('p_value', axis=0, ascending=True).reset_index(drop=True)        
        

test_stats = reduce_significance(hash_table, normal_vars)       


HASH TABLE
Ttest_indResult(statistic=array([-2.25686203]), pvalue=array([0.02412432]))
THIS IS THE data
[(0, 1, 1024), (0, 2, 2048), (0, 3, 3072), (0, 4, 4096), (0, 5, 5120), (0, 6, 6144), (0, 7, 7168), (0, 8, 8192), (0, 9, 9216), (0, 10, 10240), (0, 11, 11264), (0, 12, 12288), (0, 13, 13312), (0, 14, 14336), (0, 15, 15360), (0, 16, 16384), (0, 17, 17408), (0, 18, 18432), (0, 19, 19456), (1, 2, 2050), (1, 3, 3075), (1, 4, 4100), (1, 5, 5125), (1, 6, 6150), (1, 7, 7175), (1, 8, 8200), (1, 9, 9225), (1, 10, 10250), (1, 11, 11275), (1, 12, 12300), (1, 13, 13325), (1, 14, 14350), (1, 15, 15375), (1, 16, 16400), (1, 17, 17425), (1, 18, 18450), (1, 19, 19475), (2, 3, 3078), (2, 4, 4104), (2, 5, 5130), (2, 6, 6156), (2, 7, 7182), (2, 8, 8208), (2, 9, 9234), (2, 10, 10260), (2, 11, 11286), (2, 12, 12312), (2, 13, 13338), (2, 14, 14364), (2, 15, 15390), (2, 16, 16416), (2, 17, 17442), (2, 18, 18468), (2, 19, 19494), (3, 4, 4108), (3, 5, 5135), (3, 6, 6162), (3, 7, 7189), (3, 8, 8216), (3, 9, 92

> 8. In the cell below, create a filter for pair test cases which are significant and save these cases in a data frame. 
> 9. Print the number (len) of significant results.
> 10. Print the rows with the significant test results.

In [15]:
significance_level = 0.05
test_stats = reduce_significance(hash_table, normal_vars) 
## Put your code below. 
print(test_stats)
sig_filter_arr = test_stats.loc[test_stats['sig']=='SIG']
df_sig = pd.DataFrame(sig_filter_arr)
print(df_sig)
print(test_stats.head(10))

print('9. The length of the Significant Results are: ' + str(test_stats.loc[test_stats['sig']=='SIG'].shape[0]))


print('10. The Significant Test Results are as Follows:')
print(test_stats.loc[test_stats['sig']=='SIG'])



HASH TABLE
Ttest_indResult(statistic=array([-3.42583047]), pvalue=array([0.00062532]))
len(hash_table): 190
len(test_results): 190
len(tt_arr): 190
len(pv_arr): 190
len(sig_arr): 190
Sig-Count: 22
PRINT TEST_RESULTS
      0   1      2  t_statistic   p_value      sig
0     0   1   1024    -3.227865  0.001267      SIG
1     0   2   2048    -2.212194  0.027066      SIG
2     0   3   3072    -2.321463  0.020362      SIG
3     0   4   4096    -3.311203  0.000946      SIG
4     0   5   5120    -2.872598  0.004114      SIG
5     0   6   6144    -2.624430  0.008746      SIG
6     0   7   7168    -2.681570  0.007388      SIG
7     0   8   8192    -3.705426  0.000217      SIG
8     0   9   9216    -1.680344  0.093047  Not-Sig
9     0  10  10240    -2.462400  0.013885      SIG
10    0  11  11264    -1.297003  0.194780  Not-Sig
11    0  12  12288    -2.031263  0.042360      SIG
12    0  13  13312    -2.903111  0.003735      SIG
13    0  14  14336    -2.294894  0.021842      SIG
14    0  15  15360 

> Notice the large number of apparently significant tests. Answer the following questions:  
> 1. Is the number of false positive cases higher than expected?    
> 2. Examine which of the iid Normal vectors contribute to the false positive results. Are there vectors which contribute multiple times?   
> **End of exercise.**

Yes. There seems to be an abundant number of possible False Positives. Column vector '0' seems to contribute to alot of these Significant cases.

### Bonferroni correction  

Several adjustments to the multiple comparisons problem have been proposed. In Dunn published a method know as the **Bonferroni correction** in 1961. The Bonferroni correction is a widely used method to reduce the false positive rate of hypothesis tests.  The adjustment is simple:
$$\alpha_b = \frac{\alpha}{m}\\
with\\ 
m =\ number\ of\ groups$$

Can the Bonferroni correction help? Yes, by greatly increasing the confidence level required for a statistically significant result. The problem with the Bonferroni correction is the reduction in power as the  grows smaller. For big data problems with large numbers of groups, this issue can be especially serious. 

**Exercise 1-4:** You will now apply the Bonferroni correction to the iid Normal vectors. To do so, you will compute the Bonferroni threshold and the apply it to the p-values:   

In [31]:
## Exercise 1-4        
## m = number of groups
## alpha = confidence = 0.05

def reduce_bon_significance(hash_table, values,bon_alpha):  
    ## Create a data framreturn the results of the 
    ## the reduce process. The results are grouped by the first 
    ## index i. 
    print('HASH TABLE')
    ##print(hash_table)
   
    ncols = hash_table.shape[1]
    nrows = hash_table.shape[0]

    tt =ttest_ind(normal_vars[:,0], nr.normal(loc = 0.01, size=(1000,1)))
    print( tt)

    test_results = pd.DataFrame(columns=['i','j'])

    count2 = 0

    extract_i = []
    extract_j = []
    ##        count2 += 1
    i_arr = []
    j_arr = []
    data = []
    hash_arr  = []
    tt_arr = []
    pv_arr = []
    sig_arr = []
    sig_count = 0
    ## ^^^
    ##print(' NCOLUMNS =' + str(ncolumns))
    ## 
    for i,j in combinations(range(ncolumns), 2):
        i_arr.append(i)
        j_arr.append(j)
        
        hash_arr = hash_function(i,j)
        
        data.append((i,j,hash_function(i,j)))
        
        tt = ttest_ind(values[:,i] , values[:,j]).statistic
        ##print(tt)
        tt_arr.append(tt)
        pv = ttest_ind(values[:,i] , values[:,j]).pvalue
        
        if pv <= bon_alpha:
            sig_arr.append('SIG')
            sig_count += 1
        else:
            sig_arr.append('Not-Sig')
        pv_arr.append(pv)
 
    #test_results = pd.DataFrame(data)
    for hash in range(hash_table.shape[0]): 
        if not np.isnan(hash_table.iloc[hash,0]):
            h = 0
     
    
    #print('len(hash_table): ' + str(len(hash_table)) )  
    #print('len(test_results): ' + str(len(test_results)) ) 
    #print('len(tt_arr): '+ str(len(tt_arr)))
    #print('len(pv_arr): '+ str(len(pv_arr)))
    #print('len(sig_arr): '+ str(len(sig_arr)))
    print('Sig-Count: ' + str(sig_count))
    test_results = pd.DataFrame(data)
    test_results.columns=['i','j','hash']
    test_result.set_index('hash')
    test_results.loc[:,'t_statistic'] = tt_arr
    test_results.loc[:,'p_value'] = pv_arr
    test_results.loc[:,'sig'] = sig_arr
    print('PRINT TEST_RESULTS')
    print(test_results)
   #test_results.sort_values('p_value', axis=0, ascending=True# , inplace=True)
    return test_results.sort_values('p_value', axis=0, ascending=True).reset_index(drop=True)        
        
alpha = .05
m = 20
bon_nu_alpha = alpha/m
print('bon_nu_alpha: ' + str(bon_nu_alpha))
bon_test_stats = reduce_bon_significance(hash_table, normal_vars,bon_nu_alpha) 
##print(bon_test_stats.loc[])
##test_stats = reduce_significance(hash_table, normal_vars)
bon_test_stats.index('hash')
bon_test_stats.loc[bon_test_stats['sig'] == 'SIG']
##print(test_stats.loc[test_stats['sig']=='SIG'])
#print(df_bon_mt.loc[df_bon_mt['sig']=='SIG'])


bon_nu_alpha: 0.0025
HASH TABLE
Ttest_indResult(statistic=array([-3.02902532]), pvalue=array([0.00248487]))
Sig-Count: 4


NameError: name 'test_result' is not defined

> Even with the Bonferroni correction we have some false significance tests, if only just barely!    
> **End of exercise.**

But, can we detect small effect with Bonferroni correction, as this method significantly reduces power of tests? Execute the code in the cell below, which compares a standard Normal to a Normal with a small mean (effect size), to find out. 

In [ ]:
nr.seed(567)
print(significance_bonferroni)
ttest_ind(normal_vars[:,0], nr.normal(loc = 0.01, size=(1000,1)))

Given the Bonferroni correction, this difference in means would not be found significant. This illustrates the downside of the correction, which may prevent detection of significant effects, while still finding false significance. 

## False Discovery Rate Control Methods 

We have seen the potential pitfalls of multiple hypothesis testing. Further, we have seen that a simple approach to **false discovery rate (FDR) control** is not effective. You will now apply more sophisticated FDR control methods to control the FDR. 

Inflammatory bowel disease is an auto immune disease that is characterized by chronic inflammation in the digestive tract. In 2020, there were around 2.5 million people with inflammatory bowel disease in the United States. It is estimated that the prevalence of IBD among U.S. population will rise to around 3.5 million by 2030.There are two forms of IBD: Ulcerative Colitis (UC) and Crohn’s disease (CD). 

The specific problem we will explore is to determine which genes lead to expression of a certain disease. In this example, there are gene expression data for 97 patients. Some of these patients have ulcerative colitis and others have Crohn's disease, which are believed to be genetically inherited.    

One approach to this problem is to perform hypothesis tests on the expression of the genes between patients with the two conditions. Since there are over 10,000 genes there is considerable chance for false discovery. Therefore, careful application of FDR control is required.

To continue with the example, execute the code in the cell below to load the data and print the dimensionality of the data frame. 

In [ ]:
gene_data = pd.read_csv('../data/ColonDiseaseGeneData-Cleaned.csv')
print(gene_data.shape)

There are data from 97 patients for 10,497 genes. A large number of hypothesis tests are required!     

Execute the code in the cell below to view the first 5 columns of the data frame, which includes the expression of the first 4 genes.  

In [ ]:
 print(gene_data.iloc[:,:5])      

### Holm's method

You will apply two FDR control methods to these data.These methods attempt to conod trol the FDR while not being overly conservative like the Bonferronic correction. The first of these Holm's method.    

The Holm's method operates on the ordered set of p-values, $D = \{ p_{(1)}, p_{(2)}, p_{(3)}, \ldots, p_{(n)} \}$. The threshold for the $ith$ p-value, $p(i) is:  

$$p(i) \le Threshold(Holm's) = \frac{\alpha}{N - i + 1}$$

For example: for the 10th ordered p-value with 1,000 total tests (genes) and significance level of 0.05, the cutoff is:   

$$p(10) \le \frac{0.05}{1000 - 10 + 1} = 0.00005045$$

### Map process  

> **Exercise 01-4:** To start the processing of these data you will first create and execute code for a map process. The map process groups the data by the patient's disease into data frame, ulcerative, crohns. The keys for each of these key-value pairs are the gene identifier. Notice that one key is all that is needed in this case. Now do the following to create and execute a function, `map_gene`:   
> 1. Create a logical mask and group the values by `Disease State` into two data frames.
> 2. Return the transpose of the two data frames, removing the `Disease State` values. The result of this operation should be data frames with gene expressions in the columns and the gene identifier as the row index. 

In [ ]:
def map_gene(gene_data):  
    ## First, separate the columns by disease type  
    ## Put your code below. 
    
    
    
    
    
ulcerative, crohns = map_gene(gene_data)    

> 3. Execute the code in the cells below to display the heads of these data frames and examine the results. 

In [ ]:
ulcerative.head()

In [ ]:
crohns.head()

### Reduce process 

> **Exercise 01-5:** With the key-value pairs organized by disease state, it is time to create and execute code of a reduce process. The reduce process will compute the pairwise t-statistics and p-values for each gene and return the sorted results. Specifically, your `gene_test` with arguments of the two mapped data frames will do the following:   
> 1. Create an empty data frame with columns gene, t_statistics, and p-value.
> 2. A for loop iterates over the keys of either of the data frames.  
> 3. Compute the t-statistic and p-value for the gene (key).
> 4. Append the results to the data frame.   
> 5. Sort the results data frame, inplace, into ascending order.
> 6. Return the resulting data frame.

In [ ]:
def gene_test(ulcerative, crohns):  
    test_results = pd.DataFrame(columns=['gene','t_statistic','p_value'])
    ## Put your code below. 
    
    
    
    
    return test_results.set_index('gene')
    
gene_statistics = gene_test(ulcerative, crohns)    

### Significance of results 

With the gene data reduced to the t-test statistics, you will now determine the significance of these tests. It is important to understand that scientists believe that expression of a disease, like Corhn's, is only in a small number of genes.  

> **Exercise 01-6:** As a first step in understanding the gene expression significance complete and execute the code in the cell below to find the number of 'significant' genes using the simple single hypothesis test cutoff criteria.  

In [ ]:
significance_level =0.05
## Put your code below. 



> Does this large number of 'statistically significant' results appear credible, given that only a few genes are thought to have significant expression for this disease?    
> **End of exercise.**

> **Exercise 01-7:** We have already seen that the Bonferroni correction is a rather conservative approach to testing the significance of large numbers of hypotheses. You will now use the Bonferroni correction to test the significance of the gene expression, by completing the code in the cell below.  

In [ ]:
## Put your code below. 


> The foregoing result seems reasonable, but is it too conservative?

> **Exercise 01-08:** You will now apply the Holms method to determining significance of the gene expression test results. In the cell below complete the `holms_significance` function with arguments of the results data frame and the significance level. This function does the following:  
> 1. Find the number of test results and compute the numerator used for the cutoff calculation. 
> 2. Compute the vector of thresholds using the Holms formula. Use the Python `range`function to get the values of the index i. But, keep in mind that range produces a zero-indexed iterator, and the algorithm needs a one-indexed list.  Use the [numpy.divide](https://numpy.org/doc/stable/reference/generated/numpy.divide.html) function to perform the vector divide. Save these threshold values in a data frame in a 'holms_threshold' column.   
> 3. Using the threshold values compute a logical vector and save it in a column names 'significance' in the data frame.
> 4. Return the data frame.
> Finally, execute the function and save the results in a data frame. Then find the length of the subset where the 'significance' value is True. 

In [ ]:
def holms_significance(test_results, significance):
    ## First compute the thresholds for each of the ordered tests
    ## Put your code below. 

    
    
    ## Now we test the significance of the ordered p-values 

    return test_results

holms_results = holms_significance(gene_statistics, significance_level)    
len(holms_results.loc[holms_results.loc[:,'significant'],:])

> Despite the general properties that the Holm's method is considered less conservative than the Bonferroni correction the results agree in this case. Does this agreement give you some confidence in the result and why?  
> **End of exercise.**

You can visualize the results of the Holm's method test. The plot has two key elements:  
1. Plot the curve of the p-values vs. the order number, i. The line is color coded by significance or not.
2. Plot the threshold line. This line is straight since the threshold is a linear function of i.

In [ ]:
def plot_significance(results, threshold):
    results['number'] = range(len(results))
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.lineplot(x='number',y=threshold, data=results, ax=ax, color='black', linewidth=0.5)
    sns.scatterplot(x='number',y='p_value', hue='significant', data=results, s=3, ax=ax)
    ax.set_title('Significance of gene expression')
    ax.set_xlabel('Gene number')
    ax.set_ylabel('p-value')
    
plot_significance(holms_results.iloc[:500,:], 'holms_threshold')    

Notice the following about this plot:  
1. The p-value significance line crosses the threshold point at an apparent break point.   
2. The significant p-values are all very small since there are so many tests.

### Benamini-Hochberg FDR Control 

The Benamini-Hochberg FDR control algorithm is another way to control false discoveries. Stat with an ordered set of $n$ p-values, $D = \{ p_{(1)}, p_{(2)}, p_{(3)}, \ldots, p_{(n)} \}$ we define a false discovery rate, $q$:

$$FDR(D) \le q$$

The cutoff threshold for the ith p-value is then:
$$p_{(i)} \le Threshold(D_q) = \frac{q}{n} i$$

> **Exercise 01-9:** In this exercise you will apply the Benamini-Hochberg FDR control algorithm for testing the significance of the gene expressions. The `BH_significance` function is quite similar to the Holm's method function you have already created. Given the large number of genes you must use a low false discovery rate, $0.001$, or 1 out of 1,000. 
> Execute your function, saving the result. Then print the number of significant cases. 

In [ ]:
def BH_significance(test_results, false_discovery_tollerance):
    ## First compute the thresholds for each of the ordered tests
    ## Put your code below. 

    
    
    ## Now we test the significance of the ordered p-values 

    return test_results

BH_results = BH_significance(gene_statistics, 0.001)    
len(BH_results.loc[BH_results.loc[:,'significant'],:])

> The result is similar to the first two FDR control methods. Given the false discovery parameter of 0.0001 do you think this is a reasonable result? 

Finally, execute the code in the cell below and examine the resulting plot.

In [ ]:
plot_significance(BH_results.iloc[:500,:], 'bh_threshold')    

> **Exercise 01-10**: Bonus question. Compare the plot above to the foregoing plot for Holm's method. Are the breaks in slope of the p-value curve at the crossing point with the threshold values reasonable in both cases? 

##### Copyright 2021, Stephen F. Elston. All rights reserved. 